# [Part2] 인스타그램 크롤링

- 설명

# 1. 환경 설정

In [2]:
from selenium import webdriver 
from selenium.webdriver import ActionChains 
import time
from selenium.webdriver.common.by import By
import pandas as pd

driver = webdriver.Chrome() 
driver.set_window_size(1500, 800) #브라우저 크기를 414*900으로 설정 

# ('https://www.instagram.com/explore/tags/%EB%A7%88%EC%BC%93%EC%BB%AC%EB%A6%AC/')
driver.get('https://www.instagram.com/')
# dr.get('https://www.instagram.com/') #인스타그램 주소로 웹 오픈 
time.sleep(2) #로딩 대기

## 설계

- 접속
- 로그인
- 이동
- 최근 게시글 클릭
- 해당 게시글 내 데이터 수집(작성날짜, 본문내용, 해시태그)
- 다음 게시글로 이동
- 해당 게시글 반복 수집
- 반복(500~1000000?)

- 데이터프레임 생성
- csv로 저장(작성날짜, 본문내용, 해시태그)

- try, except, if, for  활용

In [4]:
ID = 'e_mail114@naver.com'
PW = 'rhkwp121!' # 변경
driver.find_element(By.CLASS_NAME, "_2hvTZ.pexuQ.zyHYP").send_keys(ID)
driver.find_element_by_name('password').send_keys(PW)
time.sleep(1)
driver.find_element(By.CLASS_NAME, "_9AhH0").click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"._9AhH0"}
  (Session info: chrome=93.0.4577.82)


In [5]:
driver.get('https://www.instagram.com/explore/tags/%EB%A7%88%EC%BC%93%EC%BB%AC%EB%A6%AC/') # 마켓컬리로 바로 이동

In [192]:
## 1. 최근 게시글 클릭
driver.find_element(By.CLASS_NAME, "_9AhH0").click()
time.sleep(2)

# 몇개의 게시글을 볼 것인가?
count = 10 
Final_df = pd.DataFrame()


## 2. 크롤링 부분
while count != 1:
    # 내용 부분
    Content = driver.find_elements(By.CLASS_NAME, "C4VMK")[0].text
    
    # 해시태그 부분
    HASH_TAG = driver.find_elements(By.CLASS_NAME, "xil3i")
    tag_list = list()
    for i in HASH_TAG:
        tag_list.append(i.text)
    
    # 데이터 프레임
    total_df = pd.DataFrame({"CONTENT" : Content, "HASH_TAG" : tag_list})
    total_df = total_df.groupby('CONTENT')['HASH_TAG'].agg(lambda col: ','.join(col))
    new_df = pd.DataFrame(total_df).reset_index()
    
    # 병합
    Final_df = pd.concat([Final_df,new_df], axis =0)

    # 3. 다음 페이지로 넘어가기
    driver.find_element(By.CLASS_NAME, "_65Bje").click()
    count-=1
    time.sleep(1)

Final_df # 텍스트 전처리 필요

,CONTENT,HASH_TAG
0,sinon_shop\n:)\n#고메이494#치킨마크니카레 마켓컬리에 있길래 처음 먹...,"#고메이494,#치킨마크니카레"
0,seulll_table\n-\n이것은 #다시마국수\n마켓컬리에서 비빔소스랑 같이 싸...,"#다시마국수,#마켓컬리,#슬림다시마국수"
0,nnnnnnu_diet\n#210910 #점심식단 #누래_샌드위치\n#누래_점심 :...,"#210910,#점심식단,#누래_샌드위치,#누래_점심,#무화과바질샌드위치,#디디..."
0,coolishdong\nhinok @marketkurly 💜\n\n새로운 곳에 입점...,"#내돈내산,#마켓컬리,#희녹,#새벽배송,#marketkurly,#monday"
0,bbangsori\n<21.09.26.Sun> 여러분덜!!!!!!!!! #jmt #...,"#jmt,#순대국,#해장용,#마켓컬리,#강창구찹쌀진순대,#순대만순대국,#먹스타그램,..."
0,dinnerqueen\n이 시국 필수 #마켓컬리존맛밀키트추천\n\n멀리 나가서 먹을...,"#마켓컬리존맛밀키트추천,#마켓컬리,#마켓컬리추천상품,#마켓컬리밀키트,#금미옥쌀떡볶이..."
0,boongboong2_e\n#안단잼 으로 완성하는 할매니얼 마스터!👵🏻\n⠀\n쳑쳑...,"#안단잼,#쑥인절미파운드,#바오담,#저당팥잼,#더베모,#제품제공,#마켓컬리,#온더테..."


In [194]:
Final_df['CONTENT'].values[0]
Final_df['HASH_TAG'].values[0]

'sinon_shop\n:)\n#고메이494#치킨마크니카레 마켓컬리에 있길래 처음 먹었는데요!! 고형이고 뭐고 한동안 이것만 먹을듯싶네요👍🏻👍🏻\n-\n-\n2일'

In [6]:
##데이터를 저장할 Dictionary
insta_dict = {'id':[],
              'date': [],
              'like': [],
              'text': [],
              'hashtag':[]}
 
# 첫 번째 게시물 클릭 
driver.find_element(By.CLASS_NAME, "_9AhH0").click()

In [7]:
seq = 0
start = time.time()
 
while True:
    try:
        if driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow'):
            if seq % 20 == 0:
                print('{}번째 수집 중'.format(seq), time.time() - start, sep = '\t')
 
 
 
            ## id 정보 수집
            try:
                info_id = driver.find_element_by_css_selector('h2._6lAjh').text
                insta_dict['id'].append(info_id)
            except:
                info_id = driver.find_element_by_css_selector('div.C4VMK').text.split()[0]
                insta_dict['id'].append(info_id)
 
 
            ## 시간정보 수집 
            time_raw = driver.find_element_by_css_selector('time.FH9sR.Nzb55')
            time_info = pd.to_datetime(time_raw.get_attribute('datetime')).normalize()
            insta_dict['date'].append(time_info)
 
            ## like 정보 수집
            try:
                driver.find_element_by_css_selector('button.sqdOP.yWX7d._8A5w5')
                like = driver.find_element_by_css_selector('button.sqdOP.yWX7d._8A5w5').text
                insta_dict['like'].append(like)
 
            except:
                insta_dict['like'].append('영상')
 
 
 
            ##text 정보수집
            raw_info = driver.find_element_by_css_selector('div.C4VMK').text.split()
            text = []
            for i in range(len(raw_info)):
                ## 첫번째 text는 아이디니까 제외 
                if i == 0:
                    pass
                ## 두번째부터 시작 
                else:
                    if '#' in raw_info[i]:
                        pass
                    else:
                        text.append(raw_info[i])
            clean_text = ' '.join(text)
            insta_dict['text'].append(clean_text)
 
            ##hashtag 수집
            raw_tags = driver.find_elements_by_css_selector('a.xil3i')
            hash_tag = []
            for i in range(len(raw_tags)):
                if raw_tags[i].text == '':
                    pass
                else:
                    hash_tag.append(raw_tags[i].text)
 
            insta_dict['hashtag'].append(hash_tag)
 
            seq += 1
 
            if seq == 100:
                break
 
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(1.5)
            
 
        else:
            break
            
    except:
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)

0번째 수집 중	0.016951322555541992
20번째 수집 중	36.71136212348938
40번째 수집 중	73.62219786643982
60번째 수집 중	114.65969038009644
80번째 수집 중	153.93152809143066


In [8]:
Final_df = pd.DataFrame.from_dict(insta_dict) 
Final_df.head()

In [11]:
Final_df.to_csv('instagram_crawler.csv', index = False, encoding = 'utf-8-sig')